# Test
The last kernel only got us 77% accuracy on the leaderboards. With our original 84% for our training data, it shows that we are overfitting to the training data. 

I would like to improve upon the variables that I collected by creating a child variable if the child is 14 or under, remove a few variables since we are overfitting, and convert the fare to a log scale to separate out the data. I would also like to divide the fare over the number of people on the ticket. 

So take fare, divide over people with the same ticket to get fare per person. Then get the log of that fare?
Family size = SibSp + Parch
TicketFrequency = ticket counts

In [184]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

In [185]:
# Import Raw Datasets
df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [186]:
# Fill missing ages by recollecting data
df.sort_values('Name', inplace=True)

def CA(dataframe, index, age):
    dataframe.loc[index, "Age"] = age
    
indexmissing = list(df[df.Age.isnull()].index)
agelist = agelist = [48, 18, 40, 40, 37, 45, 28, 21, 18, 48, 40, 22, 34, 30, 22, 29, 26, 49, 42, 28, 48, 32, 49, 39, 39, 23, np.nan, 18, 46, 28, 19, 21, 37, 29, 45, 33, 21, 22, 43, 20, 21, 26, 29, 7, 35, 19, 46, 23, 44, 20, 22, 40, 22, 22, 41, 41, np.nan, 35, 23, 38, 23, 5, 3, 8, 12, 31, 20, 30, 21, 26, 18, 28, 29, 22, 19, 29, 24, 31, 20, 22, 19, 62, 32, 28, 25, 23, 23, 19, 28, np.nan, 28, 30, 29, 45, 4, 35, 28, 21, 22, 18, 25, 32, 22, 27, 21, 27, 18, 17, 27, 24, 16, 21, 27, 27, 21, 30, 16, 24, 39, 2, 24, np.nan, 29, 27, 27, 30, 18, 69, 45, 30, 49, 39, 30, 30, 47, 19, 5, 8, 14, 20, 18, 16, 19, 16, 22, 42, 55, 22, 40, 20, 42, 20, 37, 57, np.nan, 57, 23, 64, 48, 37, 37, 33, 20, 23, 17, 19, 66, 21, 23, 28, 43, 54, 45, 23, 45, 19, 23]
test[test.Age.isnull()].iloc[60:,[1, 2, 3, 4]]
agelist2 = [32, 48, 17, 36, 26, 24, 37, 27, 47, 32, 32, 23, 23, 31, 29, 21, 28, 25, 20, 24, 37, 20, 25, 25, 24, 26, 40, 23, 44, 59, 30, 18, 31, 36, 26, 20, 17, 10, 43, 63, 31, 29, 41, 20, 26, 25, 32, 21, 34, 8, 25, 20, np.nan, 44, 43, 31, 26, 28, 18, 20, 22, 46, 25, 23, 33, 20, 40, 25, 25, 17, 16, 10, 44, 10, 19, 21, 44, 28, 23, 64, 26, 22, 21, 23, 35, 4]
indexmissing2 = list(test[test.Age.isnull()].index)

for x in range(len(indexmissing)):
    CA(df, indexmissing[x], agelist[x])
for x in range(len(indexmissing2)):
    CA(test, indexmissing2[x], agelist2[x])
    
df.sort_values('PassengerId', inplace=True)

In [187]:
# Combine datasets
full = pd.concat([df, test], ignore_index = True)
full.Age = full.Age.fillna(method='backfill')

# Create Family Size Variable
'''Fsize = SibSp + Parch'''
full['Fsize'] = full.SibSp + full.Parch



In [188]:

# Ticket Frequency
tcountdf = pd.DataFrame(full.Ticket.value_counts())
tcountdf['TicketC'] = tcountdf['Ticket'].copy()
tcountdf['Ticket'] = tcountdf.index
full = pd.merge(full, tcountdf, how='left', on='Ticket')
full.TicketC.astype('int')

# Get FarePer
full['FarePer'] = (full.Fare + 0.01)/(full.TicketC)
full['LogFarePer'] = np.log(full.FarePer)
full['LogFarePer'] = full['LogFarePer'].fillna(value=0)


In [189]:
# Convert to category 0,   1, 2, 3, 4, 5+
full.loc[full['Fsize'] >= 5, 'cFsize'] = 'Large'
full.loc[full['Fsize'] == 0, 'cFsize'] = 'Alone'
full.loc[full['Fsize'] == 1, 'cFsize'] = 'Single'
full.loc[full['Fsize'] == 2, 'cFsize'] = 'Double'
full.loc[full['Fsize'] == 3, 'cFsize'] = '3'
full.loc[full['Fsize'] == 4, 'cFsize'] = '4'
# full.cFsize = full.cFsize.astype(dtype='category', ordered=False, categories=[0, 1, 2, 3, 4, 5])
full.cFsize = full.cFsize.astype('category')

In [190]:
cFsizeD = pd.get_dummies(full.cFsize, prefix='cFsize').iloc[:, 1:]
full = pd.concat([full, cFsizeD], axis=1)

In [191]:
# Convert strings to dummy variables
full.Sex = full.Sex.str.replace('female', '0')
full.Sex = full.Sex.str.replace('male', '1')
full.Sex = full.Sex.astype('int')

# Create Free Dummy for 0 Fare anomalys, convert 0 fare to Null
full.loc[full.Fare == 0, 'Free'] = 1
full.loc[full.Fare != 0, 'Free'] = 0
# full.loc[full.Fare == 0, 'Fare'] = 'NaN'

full['Age'] = full['Age'].astype(int)
full.loc[(full.Age <= 14), 'Minor'] = 1
full.loc[(~(full.Age <= 14), 'Minor')] = 0
full.Minor = full.Minor.astype('int')

full.loc[(full.Age >= 40), 'Old'] = 1
full.loc[(~(full.Age >= 40), 'Old')] = 0
full.Old = full.Old.astype('int')

# Pull out tickets that multiple passengers hold, create dummy variable
full.loc[full.Ticket.duplicated(keep=False), 'Group'] = 1
full.loc[~full.Ticket.duplicated(keep=False), 'Group'] = 0


# Those who are traveling alone
full.loc[(full.SibSp == 0) & (full.Parch == 0), 'Alone'] = 1
full.loc[~((full.SibSp == 0) & (full.Parch == 0)), 'Alone'] = 0

In [192]:
tempdf = pd.DataFrame(full.Ticket.value_counts().sort_index())
tempdf.reset_index(inplace=True)
# Extract integers
tempdf['ticketint'] = tempdf.iloc[:, 0].str.extract('.*?(\d*$)\s*?', expand = False)
tempdf.iloc[787, 2] = 1 # To fix the broken ticket of LINE
tempdf['ticketint'] = tempdf['ticketint'].astype(int)
# Use consecutive test
tempdf.loc[((tempdf.ticketint-1).isin(tempdf.ticketint) | (tempdf.ticketint+1).isin(tempdf.ticketint)), 'hasCons'] = 1
tempdf.loc[~((tempdf.ticketint-1).isin(tempdf.ticketint) | (tempdf.ticketint+1).isin(tempdf.ticketint)), 'hasCons'] = 0
# For matching names, pull over the results back to dataframe by testing which ticket is in that Series
full.loc[full.Ticket.isin(tempdf.loc[tempdf.hasCons == 1, 'index']), 'hasCons'] = 1
full.loc[~(full.Ticket.isin(tempdf.loc[tempdf.hasCons == 1, 'index'])), 'hasCons'] = 0

# I understand that two chunks of this code is unnecessary, 
# but again, for the sake of preserving my current way of thinking, 
# I will leave it be and optimize the process in another notebook. 


In [193]:

# Set all members in the groups with a new dummy variable for condition of other members of group

a, _ = full.shape
for i in range(a):
    if full.loc[i, 'Group'] == 1:
        groupdf = full[full.Ticket == full.loc[i, 'Ticket']] # This is one row from this full dataframe that is a group
        b, _ = groupdf.shape
        indexlist = np.array(groupdf.index)
        for j in indexlist: # This grabs the rows in the group.
            if full.loc[j, 'Sex'] == 1: # Passenger is male
                if full.loc[j, 'Survived'] == 1: # Passenger Survived
                    full.loc[indexlist[indexlist != j], 'MSG'] = 1 # Others in group gets assigned a Male Survived Group Dummy
                elif full.loc[j, 'Survived'] == 0:
                    full.loc[indexlist[indexlist != j], 'MDG'] = 1 # Others in group gets assigned a Male Dead Group Dummy
                else: # This scenario is when survived is unknown. 
                    full.loc[indexlist[indexlist != j], 'MUG'] = 1 # Others in group gets assigned a Male Unknown Group Dummy
            else: # Passenger is female
                if full.loc[j, 'Survived'] == 1: # Passenger survived
                    full.loc[indexlist[indexlist != j], 'FSG'] = 1 # Others in group gets assigned a Female Survived Group Dummy
                elif full.loc[j, 'Survived'] == 0:
                    full.loc[indexlist[indexlist != j], 'FDG'] = 1 # Others in group gets assigned a Female Dead Group Dummy
                else:   
                    full.loc[indexlist[indexlist != j], 'FUG'] = 1 # Others in group gets assigned a Female Unknown Group Dummy
                    
# Set all members with consecutive tickets with a new dummy variable for condition of statuses of other consecutive members

# Add new column with all the ticket integers
full['ticketint'] = full.loc[:, "Ticket"].str.extract('.*?(\d*$)\s*?', expand = False)
full.loc[full.Ticket == 'LINE', 'ticketint'] = 1 # Fix Line ticket integer
full['ticketint'] = full['ticketint'].astype(int) # Set ticketint to int from string

a, _ = full.shape
for i in range(a):
    if full.loc[i, 'hasCons'] == 1: # Passenger has a consecutive ticket
        consdf = full[(full.ticketint == full.loc[i, 'ticketint']+1) | (full.ticketint == full.loc[i, 'ticketint']-1)]
        indexlist = np.array(consdf.index)
        if full.loc[i, 'Sex'] == 1: # Passenger is male
            if full.loc[i, 'Survived'] == 1: # Passenger Survived
                full.loc[indexlist, 'MSC'] = 1 # Others in group gets assigned a Male Survived Group Dummy
            elif full.loc[i, 'Survived'] == 0:
                full.loc[indexlist, 'MDC'] = 1 # Others in group gets assigned a Male Dead Group Dummy
            else: # This scenario is when survived is unknown. 
                full.loc[indexlist, 'MUC'] = 1 # Others in group gets assigned a Male Unknown Group Dummy
        else: # Passenger is female
            if full.loc[i, 'Survived'] == 1: # Passenger survived
                full.loc[indexlist, 'FSC'] = 1 # Others in group gets assigned a Female Survived Group Dummy
            elif full.loc[i, 'Survived'] == 0:
                full.loc[indexlist, 'FDC'] = 1 # Others in group gets assigned a Female Dead Group Dummy
            else:   
                full.loc[indexlist, 'FUC'] = 1 # Others in group gets assigned a Female Unknown Group Dummy
                
full[['FSG', 'MUG', 'MDG', 'FDG', 'MSG', 'FUG', 
      'MDC', 'FSC', 'FDC', 'MSC', 'MUC', 'FUC']] = full[['FSG', 'MUG', 'MDG', 
      'FDG', 'MSG', 'FUG', 'MDC', 'FSC', 'FDC', 'MSC', 'MUC', 'FUC']].fillna(value=0)
full['Age'] = full['Age'].fillna(full['Age'].mean())
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())

full[['Free', 'Group', 'hasCons', 'Alone', 'FSG', 'MUG', 'MDG', 
        'FDG', 'MSG', 'FUG', 'MDC',
        'FSC', 'FDC', 'MSC', 'MUC', 'FUC', 'Alone']] = full[['Free', 'Group', 'hasCons', 'Alone', 'FSG', 'MUG', 'MDG', 
        'FDG', 'MSG', 'FUG', 'MDC',
        'FSC', 'FDC', 'MSC', 'MUC', 'FUC', 'Alone']].astype(int)


In [300]:
# FMSG - survived
# FMSC - survived
# FMDG - nothing
# FMDC - slightly likely to die

# FFSG - likely to survive
# FFSC - likely to survive
# FFDG - likely to die
# FFDC - slightly likely to survive


# MFSG - nothing
# MMSG - likely to survive
# MFDG - dead
# MMDG - likely to die
# MFSC - nothing
# MMSC - likely to survive
# MFSC - nothing
# MFDC - nothing


full.loc[(full.Sex == 0) & (full.MSG == 1), 'FMSG'] = 1
full.loc[(full.Sex == 0) & (full.MSC == 1), 'FMSC'] = 1
full.loc[(full.Sex == 0) & (full.MDG == 1), 'FMDG'] = 1
full.loc[(full.Sex == 0) & (full.MDC == 1), 'FMDC'] = 1
full.loc[(full.Sex == 0) & (full.FSG == 1), 'FFSG'] = 1
full.loc[(full.Sex == 0) & (full.FSC == 1), 'FFSC'] = 1
full.loc[(full.Sex == 0) & (full.FDG == 1), 'FFDG'] = 1
full.loc[(full.Sex == 0) & (full.FDC == 1), 'FFDC'] = 1

full.loc[(full.Sex == 1) & (full.MSG == 1), 'MMSG'] = 1
full.loc[(full.Sex == 1) & (full.MSC == 1), 'MMSC'] = 1
full.loc[(full.Sex == 1) & (full.MDG == 1), 'MMDG'] = 1
full.loc[(full.Sex == 1) & (full.MDC == 1), 'MMDC'] = 1
full.loc[(full.Sex == 1) & (full.FSG == 1), 'MFSG'] = 1
full.loc[(full.Sex == 1) & (full.FSC == 1), 'MFSC'] = 1
full.loc[(full.Sex == 1) & (full.FDG == 1), 'MFDG'] = 1
full.loc[(full.Sex == 1) & (full.FDC == 1), 'MFDC'] = 1

full['FMSG'] = full['FMSG'].fillna(value = 0).astype(int)
full['FMSC'] = full['FMSC'].fillna(value = 0).astype(int)
full['FMDG'] = full['FMDG'].fillna(value = 0).astype(int)
full['FMDC'] = full['FMDC'].fillna(value = 0).astype(int)
full['FFSG'] = full['FFSG'].fillna(value = 0).astype(int)
full['FFSC'] = full['FFSC'].fillna(value = 0).astype(int)
full['FFDG'] = full['FFDG'].fillna(value = 0).astype(int)
full['FFDC'] = full['FFDC'].fillna(value = 0).astype(int)

full['MMSG'] = full['MMSG'].fillna(value = 0).astype(int)
full['MMSC'] = full['MMSC'].fillna(value = 0).astype(int)
full['MMDG'] = full['MMDG'].fillna(value = 0).astype(int)
full['MMDC'] = full['MMDC'].fillna(value = 0).astype(int)
full['MFSG'] = full['MFSG'].fillna(value = 0).astype(int)
full['MFSC'] = full['MFSC'].fillna(value = 0).astype(int)
full['MFDG'] = full['MFDG'].fillna(value = 0).astype(int)
full['MFDC'] = full['MFDC'].fillna(value = 0).astype(int)

In [194]:
full.iloc[:, :15].head()

In [195]:
full.iloc[:, 15:30].head()

In [345]:
# Separate the full dataset that has new features into training and testing data. 
traindf = full.iloc[:891, :]
testdf = full.iloc[891:, :]

# cols = ['Age', 'Pclass', 'Sex', 'TicketC', 'Minor', 'SibSp', 'Parch', 
#         'Free', 'FSG', 'MDG', 
#         'FDG', 'MSG', 'MDC', 'MUG', 'FUG', 'MUC', 'FUC', 
#         'FSC', 'FDC', 'MSC', 'cFsize_4', 'cFsize_Alone', 'cFsize_Double', 'cFsize_Large', 'cFsize_Single']
# No Consecutive
# cols = ['Pclass', 'Sex', 'TicketC', 'Minor', 'Old', 'SibSp', 'Parch', 'LogFarePer', 'Free', 'Group', 'hasCons', 
#         'FSG', 'MSG', 'FSC', 'MSC', 'FDG', 'MDG', 'FDC', 'MDC',
#         'cFsize_4', 'cFsize_Alone', 'cFsize_Double', 'cFsize_Large', 'cFsize_Single']
cols = ['Age', 'Pclass', 'Sex', 'TicketC', 'Minor', 'Old', 'SibSp', 'Parch', 'LogFarePer', 'Free', 'hasCons',
        'FMSG', 'FMSC', 'FMDG', 'FMDC', 'FFSG', 'FFSC', 'FFDG', 'FFDC', 'MMSG', 'MMSC', 'MMDG', 'MMDC', 'MFSG', 'MFSC', 'MFDG', 'MFDC', 
        'cFsize_4', 'cFsize_Alone', 'cFsize_Double', 'cFsize_Large', 'cFsize_Single']


test = testdf.loc[:, cols]
features = traindf.loc[:, cols]
labels = traindf.loc[:, 'Survived']

In [355]:
# Splitters
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split

# Optimizer
from sklearn.model_selection import GridSearchCV

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Metrics
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import cross_validate

In [347]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.3, random_state=42)

In [348]:
clf = DecisionTreeClassifier()
clf = AdaBoostClassifier(clf, random_state=42)
clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print(accuracy_score(pred, labels_test))
cross_validate(clf, features, labels, return_train_score=False, cv=10)

In [357]:
clf = DecisionTreeClassifier()
clf = AdaBoostClassifier(clf, random_state=42)
clf.fit(features, labels)
pred = clf.predict(test)

clf = LogisticRegression()
clf.fit(features, labels)
pred2 = clf.predict(test)

clf = RandomForestClassifier()
clf = AdaBoostClassifier(clf, random_state=42)
clf.fit(features, labels)
pred3 = clf.predict(test)

clf = KNeighborsClassifier()
clf.fit(features, labels)
pred4 = clf.predict(test)

clf = SVC()
clf.fit(features, labels)
pred5 = clf.predict(test)

In [358]:
clfguess = pd.concat([pd.Series(pred), pd.Series(pred2), pd.Series(pred3), pd.Series(pred4), pd.Series(pred5)], axis=1)

In [ ]:
testdf.iloc[192,:]

In [359]:
clfsum = pd.DataFrame(clfguess.sum(axis=1), columns=['Guess'])

In [360]:
clfsum.head()

In [361]:
clfsum.loc[clfsum.Guess <= 2, 'pred'] = 0
clfsum.loc[clfsum.Guess > 2, 'pred'] = 1

In [362]:
clfsum['pred'] = clfsum.pred.astype(int)

In [363]:
my_submission = pd.DataFrame({'PassengerId': testdf['PassengerId'], 'Survived': np.array(clfsum['pred'])})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [364]:
my_submission